In [1]:
import os
import requests
import earthaccess
import pandas as pd
import datetime as dt
import geopandas
from shapely.geometry import MultiPolygon, Polygon, box

In [2]:
doi = '10.5067/EMIT/EMITL2ARFL.001'# EMIT L2A Reflectance

# CMR API base url
cmrurl='https://cmr.earthdata.nasa.gov/search/' 

doisearch = cmrurl + 'collections.json?doi=' + doi
concept_id = requests.get(doisearch).json()['feed']['entry'][0]['id']
print(concept_id)

C2408750690-LPCLOUD


In [3]:
# Temporal Bound - Year, month, day. Hour, minutes, and seconds (ZULU) can also be included 
start_date = dt.datetime(2022, 9, 3)
end_date = dt.datetime(2022, 9, 3, 23, 23, 59)  

# CMR formatted start and end times
dt_format = '%Y-%m-%dT%H:%M:%SZ'
temporal_str = start_date.strftime(dt_format) + ',' + end_date.strftime(dt_format)
print(temporal_str)

2022-09-03T00:00:00Z,2022-09-03T23:23:59Z


In [7]:
# Search using a Polygon
polygon = geopandas.read_file('isla_gaviota.geojson')
geojson = {"shapefile": ("isla_gaviota.geojson", polygon.geometry.to_json(), "application/geo+json")}

page_num = 1
page_size = 2000 # CMR page size limit

granule_arr = []

while True:
    
     # defining parameters
    cmr_param = {
        "collection_concept_id": concept_id, 
        "page_size": page_size,
        "page_num": page_num,
        "temporal": temporal_str,
        "simplify-shapefile": 'true' # this is needed to bypass 5000 coordinates limit of CMR
    }

    granulesearch = cmrurl + 'granules.json'
    response = requests.post(granulesearch, data=cmr_param, files=geojson)
    granules = response.json()['feed']['entry']
       
    if granules:
        for g in granules:
            granule_urls = ''
            granule_poly = ''
                       
            # read granule title and cloud cover
            granule_name = g['title']
            cloud_cover = g['cloud_cover']
    
            # reading bounding geometries
            if 'polygons' in g:
                polygons= g['polygons']
                multipolygons = []
                for poly in polygons:
                    i=iter(poly[0].split (" "))
                    ltln = list(map(" ".join,zip(i,i)))
                    multipolygons.append(Polygon([[float(p.split(" ")[1]), float(p.split(" ")[0])] for p in ltln]))
                granule_poly = MultiPolygon(multipolygons)
            
            # Get https URLs to .nc files and exclude .dmrpp files
            granule_urls = [x['href'] for x in g['links'] if 'https' in x['href'] and '.nc' in x['href'] and '.dmrpp' not in x['href']]
            # Add to list
            granule_arr.append([granule_urls, cloud_cover, granule_poly])
                           
        page_num += 1
    else: 
        break
 
print(granule_arr)

[[['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFLUNCERT_001_20220903T163129_2224611_012.nc', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_MASK_001_20220903T163129_2224611_012.nc'], '82', <MULTIPOLYGON (((-62.089 -39.242, -62.512 -39.943, -61.66 -40.457, -61.237 -...>]]


In [8]:
# creating a pandas dataframe
cmr_results_df = pd.DataFrame(granule_arr, columns=["asset_url", "cloud_cover", "granule_poly"])
# Drop granules with empty geometry - if any exist
cmr_results_df = cmr_results_df[cmr_results_df['granule_poly'] != '']
# Expand so each row contains a single url 
cmr_results_df = cmr_results_df.explode('asset_url')
# Name each asset based on filename
cmr_results_df.insert(0,'asset_name', cmr_results_df.asset_url.str.split('/',n=-1).str.get(-1))

cmr_results_df

,asset_name,asset_url,cloud_cover,granule_poly
0,EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,82,"MULTIPOLYGON (((-62.0887375 -39.242054, -62.51..."
0,EMIT_L2A_RFLUNCERT_001_20220903T163129_2224611...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,82,"MULTIPOLYGON (((-62.0887375 -39.242054, -62.51..."
0,EMIT_L2A_MASK_001_20220903T163129_2224611_012.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,82,"MULTIPOLYGON (((-62.0887375 -39.242054, -62.51..."
